In [1]:
import pandas as pd
import math
from typing import List
from datetime import datetime
from sqlalchemy import MetaData, Table, create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
import os

In [3]:
engine = create_engine('sqlite:///pare.db')

In [4]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'config'))

In [5]:
from config.params import DEBUG

In [6]:
from scripts.utils import random_item, random_date, random_date_between, random_int, years_between

In [7]:
from models.partner import Partner

In [8]:
kobo_df = pd.read_csv("./inputs/kobo_local.csv", encoding="latin-1")

In [9]:
kobo_df.head()

,Departements,Communes,Section communale/Ville,Communauté/Addresse,Coordonnée GPS,_Coordonnée GPS_latitude,_Coordonnée GPS_longitude,_Coordonnée GPS_altitude,_Coordonnée GPS_precision,Nom de la compagnie,...,Nom de la personne en charge,Sexe de la personne en charge,Année de naissance de la personne en charge,age_de_la_personne_en_charge,Nom de la personne de contact,Téléphone 1 - de la personne de contact,Téléphone 2 - de la personne de contact,Email,Email2,Email3
0,Nord,Quartier-Morin,2e,Carrefour La Mort,19.7677777 -72.1944103 -28.200000762939453 4.931,19.767778,-72.194410,-28.200001,4.931,A&P Entreprise Agro Vétérinaire,...,Ing Alexis Reno,Homme,NaN,NaN,Alexis Reno,42079491.0,NaN,NaN,NaN,NaN
1,Centre,Hinche,Aguaedionne rive gauche,Coladère,19.2359631 -72.0194195 301.0 4.954,19.235963,-72.019419,301.000000,4.954,HEEMA ENGINEERING &SERVICES,...,Georges Clautain,Homme,1/1/1975,49.0,Georges Clautain,509422270.0,509363657.0,heemahaiti09@gmail.com,NaN,NaN
2,Nord,Plaine-du-Nord,"2eme section, basse plaine du Nord","10, Breda II, deuxieme section, basse plaine",19.6872993 -72.2628411 0.0 3900.0,19.687299,-72.262841,0.000000,3900.000,Este-agriv,...,Landis Lucce,Femme,NaN,NaN,Landia Lucce,37865594.0,38181832.0,NaN,NaN,NaN
3,Centre,Hinche,Juanaria,Des lattes prolongée,19.1376199 -72.0199641 237.1999969482422 2000.0,19.137620,-72.019964,237.199997,2000.000,Ferme Petit Frère de L'incarnation,...,Accilien Joseph,Homme,1/1/1959,65.0,Accilien Joseph,33582388.0,48141329.0,NaN,NaN,NaN
4,Centre,Hinche,Juanaria,Salmory,19.1180404 -72.0417987 273.0 12.833,19.118040,-72.041799,273.000000,12.833,Ferme Luckner Alexis,...,Sylvain Deblugny Arnould,Homme,1/1/1985,39.0,Sylvain Deblugny Arnould,43758477.0,37957488.0,sdeblugny1985@gmail.com,NaN,NaN


In [10]:
kobo_df['temp_created_date'] = pd.to_datetime(kobo_df['Année de fondation'])
kobo_df['created_year'] = [d.year for d in kobo_df['temp_created_date']]

In [11]:
#kobo_df['created_year']

In [12]:
#c_df.to_sql('temp_communities', con=engine, if_exists='replace', index=False)
communities = pd.read_sql('temp_communities', con=engine)

In [13]:
communities.head()

,community_id,community_name,section_name,commune_name,department_name
0,1,Trois Bois Pin,Cerca-Carvajal,Cerca-Carvajal,Centre
1,2,Bois Couleuvre,Rang,Cerca-Carvajal,Centre
2,3,Marinette,Rang,Cerca-Carvajal,Centre
3,4,Nan Guerrier/ Trois Boipins,Rang,Cerca-Carvajal,Centre
4,5,Trois Boipins,Rang,Cerca-Carvajal,Centre


In [14]:
kobo_df.columns

Index(['Departements', 'Communes', 'Section communale/Ville',
       'Communauté/Addresse', 'Coordonnée GPS', '_Coordonnée GPS_latitude',
       '_Coordonnée GPS_longitude', '_Coordonnée GPS_altitude',
       '_Coordonnée GPS_precision', 'Nom de la compagnie', 'Acronyme',
       'Année de fondation', 'age_de_l_entreprise',
       'Nom de la personne en charge', 'Sexe de la personne en charge',
       'Année de naissance de la personne en charge',
       'age_de_la_personne_en_charge', 'Nom de la personne de contact',
       'Téléphone 1 - de la personne de contact',
       'Téléphone 2 - de la personne de contact', 'Email', 'Email2', 'Email3',
       'temp_created_date', 'created_year'],
      dtype='object')

In [15]:
kobo_dict = kobo_df.to_dict(orient='records')

In [16]:
#Index(['Departements', 'Communes', 'Section communale/Ville',
#       'Communauté/Addresse', 'Coordonnée GPS', '_Coordonnée GPS_latitude',
#       '_Coordonnée GPS_longitude', '_Coordonnée GPS_altitude',
#       '_Coordonnée GPS_precision', 'Nom de la compagnie', 'Acronyme',
#       'Année de fondation', 'age_de_l_entreprise',
#       'Nom de la personne en charge', 'Sexe de la personne en charge',
#       'Année de naissance de la personne en charge',
#       'age_de_la_personne_en_charge', 'Nom de la personne de contact',
#       'Téléphone 1 - de la personne de contact',
#       'Téléphone 2 - de la personne de contact', 'Email'],
#      dtype='object')

In [17]:
def get_community(tab: str, community_name: str, section_name: str, commune_name: str) -> dict:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT * FROM " + tab + " WHERE community_name = :community_name and section_name = :section_name and commune_name = :commune_name"), 
                                                  {"section_name": section_name, "community_name": community_name, "commune_name": commune_name})
        row = result.fetchone()
        if row:
            return dict(row._mapping)  # Use _mapping to convert the row to a dictionary
        return None

In [19]:
partner_list = []
i = 1
for el in kobo_dict:
    com = get_community('temp_communities', el['Communauté/Addresse'], el['Section communale/Ville'], el['Communes'])
    email = el['Email']
    phone_1 = el['Téléphone 1 - de la personne de contact']
    phone_2 = el['Téléphone 2 - de la personne de contact']

    created_year = el['created_year']
    leader_age = el['age_de_la_personne_en_charge']

    if isinstance(leader_age, float):
        leader_age = 0

    if isinstance(created_year, float):
        created_year = 0
    else:
        created_year = int(created_year)
    if isinstance(email, float):
        email = ''
    if isinstance(phone_1, float):
        phone_1 = ''
    if isinstance(phone_2, float):
        phone_2 = ''
    partner_list.append(
        Partner(
            id=i,
            name=str(el['Nom de la compagnie']),
            acronym =str(el['Acronyme']),
            address=str(com['community_name']),
            email=email,
            phone_1=phone_1,
            phone_2=phone_2,
            created_in = created_year,
            leader_name = el['Nom de la personne en charge'],
            leader_age = leader_age,
            leader_sex = el['Sexe de la personne en charge'],
            community_id = com['community_id']
        )
    )
    i = i + 1

In [20]:
partner_list

[Partner(id=1, name='A&P Entreprise Agro Vétérinaire', acronym='A&P', address='Carrefour La Mort', email='', phone_1='', phone_2='', created_in=0, leader_name='Ing Alexis Reno', leader_age=0, leader_sex='Homme', nb_goat=None, nb_cattle=None, nb_sheep=None, nb_poultry=None, gps_lat=None, gps_lng=None, date_joined=None, community_id=78),
 Partner(id=2, name='HEEMA ENGINEERING &SERVICES', acronym='HEEMA', address='Coladère', email='heemahaiti09@gmail.com', phone_1='', phone_2='', created_in=0, leader_name='Georges Clautain', leader_age=0, leader_sex='Homme', nb_goat=None, nb_cattle=None, nb_sheep=None, nb_poultry=None, gps_lat=None, gps_lng=None, date_joined=None, community_id=22),
 Partner(id=3, name='Este-agriv', acronym='nan', address='10, Breda II, deuxieme section, basse plaine', email='', phone_1='', phone_2='', created_in=0, leader_name='Landis Lucce', leader_age=0, leader_sex='Femme', nb_goat=None, nb_cattle=None, nb_sheep=None, nb_poultry=None, gps_lat=None, gps_lng=None, date_jo